Saaksshi Jilhewar - sjilhewa \
Gauri Dalwankar - gdalwank

In [1]:
#importing libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras 

import datetime
from tensorflow import metrics

import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

2022-11-28 16:54:23.913930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 16:54:27.946209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 16:54:27.946245: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-28 16:54:37.535486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
#creating spark session

findspark.init()
findspark.find()

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MQTT") \
    .getOrCreate()

spark = SparkSession.builder.appName("mqttProject").getOrCreate()
sc    = spark.sparkContext

sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/28 16:54:51 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/11/28 16:54:51 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/11/28 16:54:51 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/11/28 16:54:51 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from matplotlib import pyplot as plt

col_names = ['tcp_flags',
 'tcp_time_delta',
 'tcp_len',
 'mqtt_conack_flags',
 'mqtt_conack_flags_reserved',
 'mqtt_conack_flags_sp',
 'mqtt_conack_val',
 'mqtt_conflag_cleansess',
 'mqtt_conflag_passwd',
 'mqtt_conflag_qos',
 'mqtt_conflag_reserved',
 'mqtt_conflag_retain',
 'mqtt_conflag_uname',
 'mqtt_conflag_willflag',
 'mqtt_conflags',
 'mqtt_dupflag',
 'mqtt_hdrflags',
 'mqtt_kalive',
 'mqtt_len',
 'mqtt_msg',
 'mqtt_msgid',
 'mqtt_msgtype',
 'mqtt_proto_len',
 'mqtt_qos',
 'mqtt_retain',
 'mqtt_sub_qos',
 'mqtt_suback_qos',
 'mqtt_ver',
 'mqtt_willmsg',
 'mqtt_willmsg_len',
 'mqtt_willtopic',
 'mqtt_willtopic_len',
 'target']

nominal_cols = []

continuous_cols = ['tcp_time_delta',
 'tcp_len',
 'mqtt_conack_flags_reserved',
 'mqtt_conack_flags_sp',
 'mqtt_conack_val',
 'mqtt_conflag_cleansess',
 'mqtt_conflag_passwd',
 'mqtt_conflag_qos',
 'mqtt_conflag_reserved',
 'mqtt_conflag_retain',
 'mqtt_conflag_uname',
 'mqtt_conflag_willflag',
 'mqtt_dupflag',
 'mqtt_kalive',
 'mqtt_len',
 'mqtt_msgid',
 'mqtt_msgtype',
 'mqtt_proto_len',
 'mqtt_qos',
 'mqtt_retain',
 'mqtt_sub_qos',
 'mqtt_suback_qos',
 'mqtt_ver',
 'mqtt_willmsg',
 'mqtt_willmsg_len',
 'mqtt_willtopic',
 'mqtt_willtopic_len']

In [4]:
to_drop =  ["mqtt_conflag_cleansess","mqtt_proto_len","mqtt_conflag_passwd","mqtt_qos"]

In [5]:
# class OutcomeCreater_binary(Transformer): # this defines a transformer that creates the outcome column which tell if it's an attack or not
    
#     def __init__(self):
#         super().__init__()
  
#     def _transform(self, dataset):
#         label_to_binary = udf(lambda name: 0.0 if name == 'legitimate' else 1.0)
#         output_df = dataset.withColumn('outcome', label_to_binary(col('target'))).drop("target")  
#         output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
#         return output_df
    
class OutcomeCreater_multi(Transformer): # this defines a transformer that creates the outcome column ehich defines the type of attack
    
    def __init__(self):
        super().__init__()
  
    def _transform(self, dataset):
        label_to_multiple = udf(lambda name: 0.0 if name == 'legitimate' else (1.0 if name == "flood" else(2.0 if name == "dos" else(3.0 if name == "bruteforce" else(4.0 if name == "slowite" else (5.0))))))
        output_df = dataset.withColumn('outcome', label_to_multiple(col('target'))).drop("target")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df
        
class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
    
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
            
        return output_df
    
def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )
    
    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+nominal_onehot_cols
    corelated_cols_to_remove = to_drop
    
    for col_name in corelated_cols_to_remove:
        feature_cols.remove(col_name)
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack/type of attack
    stage_outcome = OutcomeCreater_multi()
#     stage_outcome = OutcomeCreater_binary()

    # Removing all unnecessary columns, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols+nominal_id_cols+
        nominal_onehot_cols + continuous_cols + ['vectorized_features'])
    
    pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    
    return pipeline 

In [ ]:
test = spark.read.csv( ("gs://dataproc-staging-us-west1-657840282044-odudoeds/test30_augmented.csv"),header=True, inferSchema= True)
train = spark.read.csv( ("gs://dataproc-staging-us-west1-657840282044-odudoeds/train70_augmented.csv"),header=True, inferSchema= True)
train = train.toDF(*(c.replace('.', '_') for c in train.columns))
test = test.toDF(*(c.replace('.', '_') for c in test.columns))

In [ ]:
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(train)

train_df = preprocess_pipeline_model.transform(train)
test_df = preprocess_pipeline_model.transform(test)

In [ ]:
train_df.printSchema()
test_df.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_protoname: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)

root
 |-- tcp_flags: string (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_protoname: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)



In [ ]:
train_df = train_df.limit(100000)
test_df = test_df.limit(10000)

In [ ]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

df_train = train_df 
df_validate,df_test = test_df.randomSplit([0.5,0.5])

df_train_pandas = df_train.withColumn('features', to_array('features')).toPandas()

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

In [ ]:
df_validate_pandas = df_validate.withColumn('features', to_array('features')).toPandas()


Traceback (most recent call last):                                  (0 + 4) / 7]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [ ]:
df_test_pandas = df_test.withColumn('features', to_array('features')).toPandas()

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                  (0 + 4) / 7]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [ ]:
x_train = tf.constant(np.array(df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(df_train_pandas['outcome'].values.tolist()))

x_validate = tf.constant(np.array(df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(df_validate_pandas['outcome'].values.tolist()))


x_test = tf.constant(np.array(df_test_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(df_test_pandas['outcome'].values.tolist()))

2022-11-28 16:57:52.039251: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-28 16:57:52.054120: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-28 16:57:52.072276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cluster-b93a-m): /proc/driver/nvidia/version does not exist
2022-11-28 16:57:52.281695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
x_cv = tf.concat([x_train,x_validate],axis=0)
y_cv = tf.concat([y_train,y_validate],axis=0)

# DNN

In [ ]:
#defining dataset for cross-validation

def prepare_kfold_data(x_cv,y_cv,k):
    data_cv = tf.concat([x_cv,tf.expand_dims(y_cv,1)],axis = 1)
    data_cv = tf.random.shuffle(data_cv)
    N_samples = x_cv.shape[0]
    N_subset = N_samples //k
    return_subsets = []

    for i in range(k):
        validation_i = tf.identity(data_cv[i*N_subset:(i+1)*N_subset,:])
        train_i_index = []
        if i > 0:
            train_i_index = train_i_index + list(range(0,i*N_subset))
        if i<k-1:
            train_i_index = train_i_index + list(range((i+1)*N_subset,N_samples))
        train_i = tf.identity(tf.gather(data_cv,train_i_index,axis = 0 ))

        return_subsets.append( (train_i[:,:-2],train_i[:,-1],validation_i[:,:-2],validation_i[:,-1]) )
    
    return return_subsets

In [ ]:
k_fold_datasets = prepare_kfold_data(x_cv,y_cv,2)

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20,30,40]))
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([4,6,8]))


with tf.summary.create_file_writer('Gauri/test').as_default():
  hp.hparams_config(
    hparams=[HP_WIDTH, HP_DEPTH],
    metrics=[hp.Metric('Accuracy')],
  )


In [ ]:
def train_test_model(hparams,logdir,k_fold_datasets):
  k = len(k_fold_datasets)
  accuracy = 0.0
  for i in range(k):
    model = keras.Sequential()
    for _ in range(hparams[HP_DEPTH]):
      model.add(keras.layers.Dense(hparams[HP_WIDTH],activation='relu'))
    model.add(keras.layers.Dense(6))
    
    model.compile(
        optimizer=keras.optimizers.SGD(),
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="Accuracy_epochs")])

    history = model.fit(k_fold_datasets[i][0], k_fold_datasets[i][1], epochs=20, verbose = 2,
    validation_data = (k_fold_datasets[i][2], k_fold_datasets[i][3]))

    accuracy += np.max(history.history["val_Accuracy_epochs"])

  return accuracy/k


In [ ]:
for hp_width in HP_WIDTH.domain.values:
  for hp_depth in (HP_DEPTH.domain.values):
    hparams = {
        HP_WIDTH: hp_width,
        HP_DEPTH: hp_depth,
    }
    run_name = f"run-WIDTH{int(hparams[HP_WIDTH])}-DEPTH{hparams[HP_DEPTH]}"
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})

    run_dir = 'Gauri/test/' + run_name
    accuracy = train_test_model(hparams,run_dir,k_fold_datasets)
    
    with tf.summary.create_file_writer(run_dir).as_default():
      hp.hparams(hparams)  # record the values used in this trial
      tf.summary.scalar("Accuracy", accuracy, step=1)

    

--- Starting trial: run-WIDTH20-DEPTH4
{'NN_width': 20, 'NN_depth': 4}
Epoch 1/20
1641/1641 - 8s - loss: 1.2311 - Accuracy_epochs: 0.5708 - val_loss: 1.0694 - val_Accuracy_epochs: 0.6031 - 8s/epoch - 5ms/step
Epoch 2/20
1641/1641 - 5s - loss: 1.0171 - Accuracy_epochs: 0.6429 - val_loss: 0.9635 - val_Accuracy_epochs: 0.6517 - 5s/epoch - 3ms/step
Epoch 3/20
1641/1641 - 5s - loss: 0.9495 - Accuracy_epochs: 0.6529 - val_loss: 0.9357 - val_Accuracy_epochs: 0.6537 - 5s/epoch - 3ms/step
Epoch 4/20
1641/1641 - 5s - loss: 0.9313 - Accuracy_epochs: 0.6539 - val_loss: 0.9206 - val_Accuracy_epochs: 0.6544 - 5s/epoch - 3ms/step
Epoch 5/20
1641/1641 - 5s - loss: 0.9191 - Accuracy_epochs: 0.6544 - val_loss: 0.9125 - val_Accuracy_epochs: 0.6550 - 5s/epoch - 3ms/step
Epoch 6/20
1641/1641 - 5s - loss: 0.9100 - Accuracy_epochs: 0.6547 - val_loss: 0.9025 - val_Accuracy_epochs: 0.6554 - 5s/epoch - 3ms/step
Epoch 7/20
1641/1641 - 5s - loss: 0.9036 - Accuracy_epochs: 0.6561 - val_loss: 0.8973 - val_Accuracy_

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Gauri/test/ --port 8004

In [ ]:
model = keras.Sequential()

def test_model(x_train,y_train,x_test,y_test):
  
    for _ in range(4):
        
        model.add(keras.layers.Dense(30,activation="relu"))
    model.add(keras.layers.Dense(6))
    
    model.compile(
        optimizer=keras.optimizers.SGD(),
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="Accuracy_epochs")])

    model.fit(x_train, y_train, epochs=20)
    
    acc_test=model.evaluate(x_test,y_test)
    
    return acc_test

In [ ]:
accuracy = test_model(x_train,y_train,x_test,y_test)

Epoch 1/20
3125/3125 [==============================] - 6s 2ms/step - loss: 1.0619 - Accuracy_epochs: 0.6334
Epoch 2/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.9289 - Accuracy_epochs: 0.6542
Epoch 3/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.9067 - Accuracy_epochs: 0.6597
Epoch 4/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8967 - Accuracy_epochs: 0.6638
Epoch 5/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8920 - Accuracy_epochs: 0.6660
Epoch 6/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8888 - Accuracy_epochs: 0.6663
Epoch 7/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8870 - Accuracy_epochs: 0.6664
Epoch 8/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8853 - Accuracy_epochs: 0.6666
Epoch 9/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.8838 - Accuracy_epochs: 0.6666
Epoch 10/20
3125/31

In [ ]:
print("accuracy of the test dataset in %: ",accuracy[1]*100)

accuracy of the test dataset in %:  65.83216786384583


# NN

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20,30]))
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([1,2]))


with tf.summary.create_file_writer('Gauri/test1').as_default():
  hp.hparams_config(
    hparams=[HP_WIDTH, HP_DEPTH],
    metrics=[hp.Metric('Accuracy')],
  )


In [ ]:
for hp_width in HP_WIDTH.domain.values:
  for hp_depth in (HP_DEPTH.domain.values):
    hparams = {
        HP_WIDTH: hp_width,
        HP_DEPTH: hp_depth,
    }
    run_name = f"run-WIDTH{int(hparams[HP_WIDTH])}-DEPTH{hparams[HP_DEPTH]}"
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})

    run_dir = 'Gauri/test1/' + run_name
    accuracy = train_test_model(hparams,run_dir,k_fold_datasets)
    
    with tf.summary.create_file_writer(run_dir).as_default():
      hp.hparams(hparams)  # record the values used in this trial
      tf.summary.scalar("Accuracy", accuracy, step=1)

    

--- Starting trial: run-WIDTH20-DEPTH1
{'NN_width': 20, 'NN_depth': 1}
Epoch 1/20
1641/1641 - 5s - loss: 1.0901 - Accuracy_epochs: 0.6283 - val_loss: 0.9987 - val_Accuracy_epochs: 0.6446 - 5s/epoch - 3ms/step
Epoch 2/20
1641/1641 - 4s - loss: 0.9857 - Accuracy_epochs: 0.6467 - val_loss: 0.9743 - val_Accuracy_epochs: 0.6474 - 4s/epoch - 3ms/step
Epoch 3/20
1641/1641 - 4s - loss: 0.9705 - Accuracy_epochs: 0.6489 - val_loss: 0.9640 - val_Accuracy_epochs: 0.6488 - 4s/epoch - 3ms/step
Epoch 4/20
1641/1641 - 4s - loss: 0.9625 - Accuracy_epochs: 0.6504 - val_loss: 0.9579 - val_Accuracy_epochs: 0.6510 - 4s/epoch - 3ms/step
Epoch 5/20
1641/1641 - 4s - loss: 0.9570 - Accuracy_epochs: 0.6513 - val_loss: 0.9523 - val_Accuracy_epochs: 0.6516 - 4s/epoch - 3ms/step
Epoch 6/20
1641/1641 - 4s - loss: 0.9523 - Accuracy_epochs: 0.6519 - val_loss: 0.9483 - val_Accuracy_epochs: 0.6523 - 4s/epoch - 3ms/step
Epoch 7/20
1641/1641 - 4s - loss: 0.9482 - Accuracy_epochs: 0.6527 - val_loss: 0.9446 - val_Accuracy_

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Gauri/test1/ --port 8017

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
model = keras.Sequential()

def test_model(x_train,y_train,x_test,y_test):
  
    for _ in range(2):
        
        model.add(keras.layers.Dense(30,activation="relu"))
    model.add(keras.layers.Dense(6))
    
    model.compile(
        optimizer=keras.optimizers.SGD(),
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="Accuracy_epochs")])

    model.fit(x_train, y_train, epochs=20)
    
    acc_test=model.evaluate(x_test,y_test)
    
    return acc_test

In [ ]:
accuracy = test_model(x_train,y_train,x_test,y_test)

Epoch 1/20
3125/3125 [==============================] - 6s 2ms/step - loss: 1.0513 - Accuracy_epochs: 0.6314
Epoch 2/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.9070 - Accuracy_epochs: 0.6561
Epoch 8/20
3125/3125 [==============================] - 6s 2ms/step - loss: 0.9007 - Accuracy_epochs: 0.6647
Epoch 9/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8967 - Accuracy_epochs: 0.6655
Epoch 10/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8935 - Accuracy_epochs: 0.6659
Epoch 11/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8914 - Accuracy_epochs: 0.6661
Epoch 12/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8896 - Accuracy_epochs: 0.6662
Epoch 13/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8884 - Accuracy_epochs: 0.6663
Epoch 14/20
3125/3125 [==============================] - 5s 2ms/step - loss: 0.8871 - Accuracy_epochs: 0.6665
Epoch 15/20
31

In [ ]:
print("accuracy of the test dataset in %: ",accuracy[1]*100)

accuracy of the test dataset in %:  65.63188433647156
